# Data Cleaning Notebook
This notebook is used to clean the datasets for global temperature anomalies and related information.

In [71]:
# Step 1: Import Required Libraries
import pandas as pd
import numpy as np

### Step 2: Load the Datasets
This step will load the raw datasets and handle the first row as a potential title.

In [74]:
def load_and_clean_csv(file_path):
    try:
        # Step 1: Try loading the file with no headers, inspect the first few rows
        print(f"Attempting to load {file_path}...")
        df = pd.read_csv(file_path, header=None, delimiter=',', on_bad_lines='skip', engine='python')
        
        # Step 2: Check if the first row contains a title (e.g., 'Land-Ocean: Global Means')
        first_row = df.iloc[0, 0]
        
        if isinstance(first_row, str) and "Land-Ocean" in first_row:
            print(f"First row contains a title in {file_path}. Skipping the first row.")
            # Reload the file, skipping the first row and using the second row as the header
            df = pd.read_csv(file_path, header=1, delimiter=',', on_bad_lines='skip', engine='python')
        else:
            print(f"First row is not a title in {file_path}. Proceeding as normal.")
            df = pd.read_csv(file_path, header=0, delimiter=',', on_bad_lines='skip', engine='python')
        return df
    except pd.errors.ParserError as e:
        print(f"Error parsing {file_path}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred while parsing {file_path}: {e}")
        return None

# Load datasets
global_monthly = load_and_clean_csv("../data/raw/Globalmonthlyandseasonal.csv")
northern_hemisphere = load_and_clean_csv("../data/raw/NorthernHemisphere.csv")
southern_hemisphere = load_and_clean_csv("../data/raw/SouthernHemisphere.csv")
zone_annual = load_and_clean_csv("../data/raw/ZoneAnnual.csv")

# Print first few rows to confirm
if global_monthly is not None:
    print(global_monthly.head())
if northern_hemisphere is not None:
    print(northern_hemisphere.head())
if southern_hemisphere is not None:
    print(southern_hemisphere.head())
if zone_annual is not None:
    print(zone_annual.head())

Attempting to load ../data/raw/Globalmonthlyandseasonal.csv...
First row contains a title in ../data/raw/Globalmonthlyandseasonal.csv. Skipping the first row.
Attempting to load ../data/raw/NorthernHemisphere.csv...
First row contains a title in ../data/raw/NorthernHemisphere.csv. Skipping the first row.
Attempting to load ../data/raw/SouthernHemisphere.csv...
First row contains a title in ../data/raw/SouthernHemisphere.csv. Skipping the first row.
Attempting to load ../data/raw/ZoneAnnual.csv...
First row is not a title in ../data/raw/ZoneAnnual.csv. Proceeding as normal.
   Year   Jan   Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   Oct   Nov  \
0  1880 -0.20 -0.25 -0.09 -0.16 -0.09 -0.22 -0.20 -0.09  -.15  -.22  -.22   
1  1881 -0.19 -0.15  0.02  0.04  0.07 -0.19  0.01 -0.04  -.16  -.22  -.18   
2  1882  0.16  0.14  0.05 -0.15 -0.13 -0.22 -0.16 -0.07  -.14  -.23  -.17   
3  1883 -0.29 -0.36 -0.12 -0.18 -0.18 -0.07 -0.07 -0.14  -.22  -.11  -.24   
4  1884 -0.12 -0.08 -0.36 -0.40 -0.

### Step 3: Handle Missing Values
Replace placeholder values such as '***' with `NaN`.

In [75]:
# Handle missing values by replacing '***' with NaN
datasets = [global_monthly, northern_hemisphere, southern_hemisphere, zone_annual]

for dataset in datasets:
    dataset.replace("***", np.nan, inplace=True)

### Step 4: Ensure Correct Data Types
Ensure the `Year` column is of integer type and check the data types of other columns.

In [76]:
for dataset in datasets:
    if 'Year' in dataset.columns:
        # Convert 'Year' to numeric, setting errors='coerce' to turn non-numeric entries into NaN
        dataset['Year'] = pd.to_numeric(dataset['Year'], errors='coerce')
        
        # Drop rows where 'Year' is NaN (i.e., where conversion failed)
        dataset.dropna(subset=['Year'], inplace=True)
        
        # Convert 'Year' to integers, using Int64 which allows for missing data if any exists
        dataset['Year'] = dataset['Year'].astype('Int64')
    
    # Print dataset info for debugging
    print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    145 non-null    Int64  
 1   Jan     145 non-null    float64
 2   Feb     145 non-null    float64
 3   Mar     145 non-null    float64
 4   Apr     145 non-null    float64
 5   May     145 non-null    float64
 6   Jun     145 non-null    float64
 7   Jul     145 non-null    float64
 8   Aug     145 non-null    float64
 9   Sep     144 non-null    object 
 10  Oct     144 non-null    object 
 11  Nov     144 non-null    object 
 12  Dec     144 non-null    object 
 13  J-D     144 non-null    object 
 14  D-N     143 non-null    object 
 15  DJF     144 non-null    object 
 16  MAM     145 non-null    float64
 17  JJA     145 non-null    float64
 18  SON     144 non-null    object 
dtypes: Int64(1), float64(10), object(8)
memory usage: 21.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex:

### Step 5: Handle Duplicates
Remove any duplicate rows in the datasets.

In [77]:
# Drop duplicate rows
for dataset in datasets:
    dataset.drop_duplicates(inplace=True)

### Step 6: Validate Year Ranges
Remove rows where the `Year` value is outside the expected range (1880-2023).

In [78]:
# Keep only rows where 'Year' is between 1880 and 2023
for i in range(len(datasets)):
    if 'Year' in datasets[i].columns:
        datasets[i] = datasets[i][(datasets[i]['Year'] >= 1880) & (datasets[i]['Year'] <= 2023)]

### Step 7: Fill Missing Values
For missing temperature data, fill the gaps using the column means.

In [79]:
# Fill missing values with column means
for dataset in datasets:
    num_cols = dataset.select_dtypes(include=[np.number]).columns
    dataset[num_cols] = dataset[num_cols].fillna(dataset[num_cols].mean())

/var/folders/sd/7ryt1xgs2sg3bxmccx2f715c0000gn/T/ipykernel_83768/373111762.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[num_cols] = dataset[num_cols].fillna(dataset[num_cols].mean())


### Step 8: Save Cleaned Datasets
Save the cleaned datasets to new CSV files.

In [80]:
# Save cleaned data
global_monthly.to_csv("../data/cleaned/cleaned_globalmonthlyandseasonal.csv", index=False)
northern_hemisphere.to_csv("../data/cleaned/cleaned_northernhemisphere.csv", index=False)
southern_hemisphere.to_csv("../data/cleaned/cleaned_southernhemisphere.csv", index=False)
zone_annual.to_csv("../data/cleaned/cleaned_zoneannual.csv", index=False)